In [88]:
import pandas as pd

df = pd.read_excel('sql-苏宁回款时间.xlsx')
df

,回款日期,店铺,订单日期
0,2021-09-30,苏宁荣耀旗舰店,2021-09
1,2021-09-30,苏宁小米旗舰店,2021-09
2,2021-09-30,苏宁华为旗舰店,2021-09
3,2021-10-30,苏宁荣耀旗舰店,2021-10
4,2021-10-30,苏宁小米旗舰店,2021-10
5,2021-10-30,苏宁华为旗舰店,2021-10
6,2021-11-30,苏宁荣耀旗舰店,2021-11
7,2021-11-30,苏宁小米旗舰店,2021-11
8,2021-11-30,苏宁华为旗舰店,2021-11


In [83]:
# -*- coding:utf-8 -*-
import time
from pymysql import *

# 装饰器，计算插入50000条数据需要的时间
def timer(func):
    def decor(*args):
        start_time = time.time()
        func(*args)
        end_time = time.time()
        d_time = end_time - start_time
        print("the running time is : ", d_time)

    return decor

@timer
def add_test_users():
    
    usersvalues = []
    for num in range(1, 50000):
        usersvalues.append(('需要插入的字段对应的value'))  # 注意要用两个括号扩起来

    conn = connect(host='127.0.0.1', port=3306, user='root', password='12345678', database='test', charset='utf8')
    cs = conn.cursor()  # 获取光标
    # 注意这里使用的是executemany而不是execute，下边有对executemany的详细说明 月份	店铺	售后费用	物流费	销售数量	每单费用	合计
    
    for i in range(0,len(df)):
        sql = "insert into 物流费 values('%s','%s','%s','%s','%s','%s','%s')"% (df.月份[i],df.店铺[i],df.售后费用[i],df.物流费[i],df.销售数量[i],df.每单费用[i],df.合计[i])
        cs.execute(sql)
        
    #.execute("INSERT INTO test (money,gui) VALUE (%d,'%s')" % (i, k))
    # executemany execute
    conn.commit()
    cs.close()
    conn.close()
    print('OK')
    # 关闭数据库连接
    cs.close()
add_test_users()

OK
the running time is :  0.01495981216430664


In [ ]:
def insql(len_df,sql_insert):
    for i in range(0,len(df)):
        sql = "insert into 物流费 values('%s','%s','%s','%s','%s','%s','%s')"% (df.月份[i],df.店铺[i],df.售后费用[i],df.物流费[i],df.销售数量[i],df.每单费用[i],df.合计[i])
        cs.execute(sql)

In [77]:
a = ['%s']*9
a


['%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s']

In [78]:
'","'.join(a)[4:-4]

'"%s","%s","%s","%s","%s","%s","%s"'

In [6]:
for i in range(0,len(df)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [7]:
f"{df.月份[0]},{df.店铺[0]},{df.物流费[0]},{df.销售数量[0]},{df.每单费用[0]},{df.合计[0]}"

'2021-09,世翔京喜自营店(京喜),20221.96,1939.0,10.563883445074781,20483.37'

In [44]:
import os
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import yaml
from tqdm import tqdm
from time import sleep


In [45]:
print('正在加载配置文件......')
def config_dp():
    # print('正在加载配置文件......')
    try:
        with open('config.yml', 'r', encoding='utf-8') as f:
            result = yaml.load(f.read(), Loader=yaml.FullLoader)
        return result
    except:
        print('缺少配置文件:config.yml')
        return None

path = config_dp()['path']
con = config_dp()
database : config_dp()['database_c']['database']
host = config_dp()['database_c']['host']
port = config_dp()['database_c']['port']
user = config_dp()['database_c']['user']
password = config_dp()['database_c']['password']
database = config_dp()['database_c']['database']
charset = config_dp()['database_c']['charset']
query_max_date = config_dp()['select']
fugai = config_dp()['fugai']
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset={charset}")

# query_max_date
# query_max_date['vc结算单']['select max(结算日期) from vc结算单']

正在加载配置文件......


In [46]:
#读取数据文件
def read_df(path,file_name,usecols=None):
    df = pd.read_excel(path+file_name,usecols = usecols)
    return df

In [56]:
#数据库添加数据 重新添加为：replace
def sql_insert(df,table_name,fun='append'):
    df.to_sql(table_name, engine, schema=database, if_exists=fun, index=False,
                chunksize=None, dtype=None)

In [48]:
#读取数据库 tables_name

def tables_list(database,sql):
    db = pymysql.connect(host=host,port=port,user=user,password=password,database=database,charset=charset)

    #2.利用db方法创建游标对象
    cur = db.cursor()

    #3.利用游标对象execute()方法执行SQL命令
    # cur.execute("show tables from test")
    cur.execute(sql)
    #fetchone()方法获取返回对象的单条数据 fetchall 方法获取返回对象的所有数据
    data = cur.fetchall()
    
    #关闭数据库连接
    db.close()
    
    return data

In [57]:
k_n = []
v_n = []
s = []
for key,value in query_max_date.items():
    for k,v in value.items():
        date = tables_list(database,k)[0][0]
        s.append(str(date))
        k_n.append(key)
        v_n.append(str(v))
df_max = pd.DataFrame(data=[k_n,v_n,s],index = ['表','链接','最大日期']).T
df_max

,表,链接,最大日期
0,vc结算单,https://vcf.jd.com/sub_finance/jxSettlerequest...,2021-12-21 00:00:00
1,京东,https://pdropship.jd.com/exportCenter/initList...,None
2,京东退货单,https://vcreturns.jd.com/returns/orderList,2021-12-21 15:59:22
3,苏宁报表,null,None
4,旺店通,null,2021-12-21 23:56:09
5,小柚旺店通销售管理,http://erp2.xyooou.com/Main,2021/12/20 21:36:12
6,自采单流水账,http://boss2.xyooou.com/Home/Index,2021/9/9 18:45:01
7,自采销售单流水账,http://boss2.xyooou.com/Home/Index,2021/9/8 11:36:26


In [58]:
file_name = os.listdir(path)
print(f'路径：{path}\n文件列表:\n'+'\n'.join(file_name))


路径：./每日/excel/
文件列表:
~$京东报表.xlsx
京东报表.xlsx
旺店通.xlsx
旺店通销售管理.xlsx
结算单单据明细.xlsx
自采单流水账.xlsx
自采销售单流水账.xlsx
非图书退货订单导出.xlsx


In [59]:
table_names = []
s_count = []
e_count = []


for key,value in tqdm(con['re'].items()):
    start = tables_list(database,"select count(*) from "+key)[0][0]
    sleep(0.05)
    for k,y in value.items():
        try:
            df = read_df(path,k+'.xlsx',y)
            if key in fugai:
                sql_insert(df,key,'replace')
            else:
                sql_insert(df,key)
            table_names.append(key)
            s_count.append(start)
            end = tables_list(database,"select count(*) from "+key)[0][0]
            e_count.append(end)
        except:
            print("没有文件"+k+'.xlsx,请检查文件是否存在，或‘后缀名’不正确')
            break

  0%|          | 0/10 [00:00<?, ?it/s]

没有文件sql-苏宁回款时间.xlsx,请检查文件是否存在，或‘后缀名’不正确
没有文件sql-物流费.xlsx,请检查文件是否存在，或‘后缀名’不正确


 30%|███       | 3/10 [00:00<00:00,  9.70it/s]

没有文件京东报表.xlsx,请检查文件是否存在，或‘后缀名’不正确


 50%|█████     | 5/10 [00:00<00:00,  7.84it/s]

没有文件sql-苏宁报表.xlsx,请检查文件是否存在，或‘后缀名’不正确


100%|██████████| 10/10 [00:13<00:00,  3.41s/it]


In [ ]:
# read_df(path,'京东报表.xlsx',usecols=['客户订单号', '商品编码', '品牌', '商品名称', '商品单价', '采购价', '采购单号', '退货单号', '购买数量', '收货人姓名', '收货人电话', '收货人地址', '省', '市', '县', '预计发货时间', '商家仓ID', '商家仓名称', '订单备注', '商家备注', '父单号', '支付方式', '出库时间', '下单时间', '可出库时间', '订单金额', '用户支付金额', '承运商', '运单号', '订单状态', '订单完成时间', '是否有退款申请'])

In [60]:
relult = pd.DataFrame(data=[e_count,s_count],columns=table_names,index = ['添加后','添加前']).T
relult = relult.assign(本次添加条数=relult.添加后-relult.添加前)
relult

,添加后,添加前,本次添加条数
vc结算单,4088,3577,511
京东退货单,520,455,65
旺店通,18328,16037,2291
小柚旺店通销售管理,7240,6335,905
自采单流水账,8163,8163,0
自采销售单流水账,23095,23095,0


In [ ]:
pd.read_excel(path+"sql-京东报表.xlsx",usecols=['客户订单号', '商品编码', '品牌', '商品名称', '商品单价', '采购价', '采购单号', '退货单号', '购买数量',
       '收货人姓名', '收货人电话', '收货人地址', '省', '市', '县', '预计发货时间', '商家仓ID', '商家仓名称',
       '订单备注', '商家备注', '父单号', '支付方式', '出库时间', '下单时间', '可出库时间', '订单金额',
       '用户支付金额', '承运商', '运单号', '订单状态', '订单完成时间', '是否有退款申请', '商家编码', '店铺']).head(3).columns
#,usecols=['客户订单号,商品编码,品牌,商品名称,商品单价,采购价,采购单号,退货单号,购买数量,收货人姓名,收货人电话,收货人地址,省,市,县,预计发货时间,商家仓ID,商家仓名称,订单备注,商家备注,父单号,支付方式,出库时间,下单时间,可出库时间,订单金额,用户支付金额,承运商,运单号,订单状态,订单完成时间,是否有退款申请,商家编码,店铺']

In [2]:
a = '销售订单编号,渠道单状态,供应商会员名,分销商会员名,消费者留言,分销单创建时间,分销单付款时间,分销单发货时间,分销单退款成功时间,分销单确认收货时间,经营模式,支付方式,渠道产品名称,产品SKU名称,商家编码,销售数量,渠道价格(元),分销金额小记(元),改价金额(元),分销金额小记汇总(元),分销邮费(元),分销单金额汇总(元),分销商已支付金额(元),备注文字,备注颜色,分销单退款状态,已退款金额(元),消费者订单编号,消费者订单支付状态,消费者支付宝交易号,分销商支付宝交易号,收货人,手机,电话,省,市,区,地址'

In [3]:
a.split(',')

['销售订单编号',
 '渠道单状态',
 '供应商会员名',
 '分销商会员名',
 '消费者留言',
 '分销单创建时间',
 '分销单付款时间',
 '分销单发货时间',
 '分销单退款成功时间',
 '分销单确认收货时间',
 '经营模式',
 '支付方式',
 '渠道产品名称',
 '产品SKU名称',
 '商家编码',
 '销售数量',
 '渠道价格(元)',
 '分销金额小记(元)',
 '改价金额(元)',
 '分销金额小记汇总(元)',
 '分销邮费(元)',
 '分销单金额汇总(元)',
 '分销商已支付金额(元)',
 '备注文字',
 '备注颜色',
 '分销单退款状态',
 '已退款金额(元)',
 '消费者订单编号',
 '消费者订单支付状态',
 '消费者支付宝交易号',
 '分销商支付宝交易号',
 '收货人',
 '手机',
 '电话',
 '省',
 '市',
 '区',
 '地址']